This notebook is part of the *orix* documentation https://orix.readthedocs.io. Links to the documentation won’t work from the notebook.

In [2]:
%matplotlib inline

import tempfile

from diffpy.structure import Atom, Lattice, Structure
import matplotlib.pyplot as plt
import numpy as np

from orix import data, io, plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d


plt.rcParams.update({"figure.figsize": (7, 7), "font.size": 15})
tempdir = tempfile.mkdtemp() + "/"

## Modify crystal phases

The phases are stored in a  [PhaseList](../reference/generated/orix.crystal_map.PhaseList.rst) instance in the `CrystalMap.phases` attribute

In [7]:
xmap.phases

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite         None          432                 432    tab:blue
 2    ferrite         None          432                 432  tab:orange

### Symmetry

The point group symmetry are stored in the vendor and EMsoft files, however they provide no space group symmetry. We can set this *per phase* by providing a space group number (1-230) according to the International Tables of Crystallography (useful link: http://img.chem.ucl.ac.uk/sgp/large/sgp.htm)

In [8]:
xmap.phases[1].space_group = 225
xmap.phases[2].space_group = 229

xmap.phases

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

Note that this also changed the point group, because this is always determined from the space group. But the proper point group, without any inversion or mirror planes, stayed the same. The `space_group` attribute stores a [diffpy.structure.spacegroups.SpaceGroup](https://www.diffpy.org/diffpy.structure/mod_spacegroup.html#diffpy.structure.spacegroupmod.SpaceGroup) instance.

We can get the point group which a space group is the subgroup of

In [9]:
print(symmetry.get_point_group(200).name, symmetry.get_point_group(230).name)

m-3 m-3m


The point group stores symmetry operations as quaternions. We can get them as orientation matrices

In [10]:
xmap.phases[1].point_group[:2]

Symmetry (2,) 
[[1.     0.     0.     0.    ]
 [0.7071 0.     0.     0.7071]]

In [11]:
xmap.phases[1].point_group[:2].to_matrix()

array([[[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[-4.26642159e-17, -1.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00, -4.26642159e-17,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]])

`diffpy.structure` stores rotation symmetry operations as orientation matrices and translations as 1D arrays

In [12]:
[(i.R, i.t) for i in xmap.phases[1].space_group.symop_list[:2]]

[(array([[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]),
  array([0., 0., 0.])),
 (array([[-1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0.,  1.]]),
  array([0., 0., 0.]))]

We can get the quaternion representation of these matrices

In [13]:
[
    Rotation.from_matrix(i.R)
    for i in xmap.phases[1].space_group.symop_list[:2]
]

[Rotation (1,)
 [[1. 0. 0. 0.]],
 Rotation (1,)
 [[0. 0. 0. 1.]]]

### Index phase list

The phase list can be indexed by phase ID or name

In [14]:
xmap.phases[1]

<name: austenite. space group: Fm-3m. point group: m-3m. proper point group: 432. color: tab:blue>

In [15]:
xmap.phases["austenite"]

<name: austenite. space group: Fm-3m. point group: m-3m. proper point group: 432. color: tab:blue>

In [16]:
xmap.phases[1:]

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

In [17]:
xmap.phases["austenite", "ferrite"]

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

When asking for a single phase, either by an integer or a single string, a [Phase](../reference/generated/orix.crystal_map.Phase.rst) instance was returned. In the other cases, a `PhaseList` object was returned

In [18]:
print(type(xmap.phases[1]), type(xmap.phases[1:]))

<class 'orix.crystal_map._phase.Phase'> <class 'orix.crystal_map._phase_list.PhaseList'>


Valid point group names to use when setting the point group symmetry are

In [19]:
[point_group.name for point_group in symmetry._groups]

['1',
 '-1',
 '211',
 '121',
 '112',
 'm11',
 '1m1',
 '11m',
 '2/m',
 '222',
 'mm2',
 'mmm',
 '4',
 '-4',
 '4/m',
 '422',
 '4mm',
 '-42m',
 '4/mmm',
 '3',
 '-3',
 '321',
 '312',
 '32',
 '3m',
 '-3m',
 '6',
 '-6',
 '6/m',
 '622',
 '6mm',
 '-6m2',
 '6/mmm',
 '23',
 'm-3',
 '432',
 '-43m',
 'm-3m']

In [20]:
xmap.phases["austenite"].point_group = "-43m"

xmap.phases

/home/arg6/GitHub/orix/orix/crystal_map/_phase.py:255: UserWarning: Setting space group to 'None', as current space group 'Fm-3m' is derived from current point group 'm-3m'
  warnings.warn(


Id       Name  Space group  Point group  Proper point group       Color
 1  austenite         None         -43m                  23    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

Note that the `space_group` was set to `None` since space group Fm-3m is not a subgroup of -43m.

Let's revert to the correct space group (and the name, for convenience)

In [21]:
xmap.phases["austenite"].space_group = 225

xmap.phases

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

We can add a phase by giving its name and point group symmetry

In [22]:
xmap.phases.add(Phase("sigma", point_group="4/mmm"))

xmap.phases

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange
 3      sigma         None        4/mmm                 422   tab:green

When adding a phase to the phase list like this, the phases' structure contains no atoms and the default lattice parameters are used

In [23]:
xmap.phases["sigma"].structure.lattice.abcABG()

(1.0, 1.0, 1.0, 90.0, 90.0, 90.0)

So let's set this

In [24]:
xmap.phases["sigma"].structure.lattice = Lattice(
    0.880, 0.880, 0.880, 90, 90, 90
)
print(xmap.phases["sigma"].structure.lattice)

Lattice(a=0.88, b=0.88, c=0.88, alpha=90, beta=90, gamma=90)


If some data points are considered as not indexed, a "not_indexed" phase can be added to the phase list to keep track of these points

In [25]:
xmap.phases.add_not_indexed()

xmap.phases

Id         Name  Space group  Point group  Proper point group       Color
-1  not_indexed         None         None                None       white
 1    austenite        Fm-3m         m-3m                 432    tab:blue
 2      ferrite        Im-3m         m-3m                 432  tab:orange
 3        sigma         None        4/mmm                 422   tab:green

No points in this data set are considered not indexed. A phase list with only the phases in the data is stored in the `phases_in_data` attribute

In [26]:
xmap.phases_in_data

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

We can of course remove a phase from the phase list, either by its name or phase ID

In [27]:
del xmap.phases["sigma"]
del xmap.phases[-1]

xmap.phases

Id       Name  Space group  Point group  Proper point group       Color
 1  austenite        Fm-3m         m-3m                 432    tab:blue
 2    ferrite        Im-3m         m-3m                 432  tab:orange

### Properties

The phase name, space group, point group, proper point group, color and structure can be accessed for the full phase list or a single phase

In [ ]:
print(xmap.phases.names)
print([i.short_name for i in xmap.phases.space_groups])
print([i.name for i in xmap.phases.point_groups])
print([i.proper_subgroup.name for i in xmap.phases.point_groups])
print(xmap.phases.colors)
print(xmap.phases.structures)

Note that the structures' representations are empty lists since no atoms have been added to them yet.

In [ ]:
xmap.phases["austenite"]
print(xmap.phases["austenite"].name)
print(xmap.phases["austenite"].space_group.short_name)
print(xmap.phases["austenite"].point_group.name)
print(xmap.phases["austenite"].point_group.proper_subgroup.name)
print(xmap.phases["austenite"].color)
print(xmap.phases["austenite"].structure)

These attributes (not the phase ID) can be set *per phase*

In [ ]:
xmap.phases["austenite"].structure = Structure(
    lattice=Lattice(0.36, 0.36, 0.36, 90, 90, 90)
)
print(xmap.phases["austenite"].structure)

xmap.phases["austenite"].color = "lime"  # Sets RGB tuple (0, 1, 0)
print(xmap.phases["austenite"].color_rgb)

xmap.phases

Valid color strings can be found here: https://matplotlib.org/stable/tutorials/colors/colors.html

#### Create phase list

We can create a phase list using [PhaseList](../reference/generated/orix.crystal_map.PhaseList.rst)

In [ ]:
PhaseList(
    names=["al", "cu"],
    space_groups=[225, 225],
    colors=["lime", "xkcd:violet"],
    ids=[0, 1],
    structures=[
        Structure(
            atoms=[Atom("al", [0] * 3)],
            lattice=Lattice(0.405, 0.405, 0.405, 90, 90, 90),
        ),
        Structure(
            atoms=[Atom("cu", [0] * 3)],
            lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90),
        ),
    ],
)

or by creating `Phase` objects and passing these to the first argument in `PhaseList.__init__()` as a list (or single `Phase` objects)

In [ ]:
al = Phase(name="al", space_group=225, color="C0")
cu = Phase(
    color="C1",
    structure=Structure(
        title="cu", lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90)
    ),
)

PhaseList([al, cu])

Note that the Cu phase name was retrieved from the `Structure` object.

### Copying

If we want a shallow copy of the phase list

In [ ]:
pl = xmap.phases
pl["ferrite"].color = "red"

xmap.phases

If we want a deep copy of the phase list

In [ ]:
pl = xmap.phases.deepcopy()
pl.add(Phase("chi", point_group="-43m"))
print(pl, "\n")

print(xmap.phases)

## Orientation data

Rotations are stored in a  [Rotation](../reference/generated/orix.quaternion.Rotation.rst) instance

In [ ]:
xmap.rotations

Orientations *per phase* can be obtained by applying the phase point group symmetry

In [ ]:
O_au = xmap["austenite"].orientations

O_au

The above is equivalent to

In [ ]:
R_au = xmap["austenite"].rotations
O_au2 = Orientation(R_au, symmetry=xmap["austenite"].phases[1].point_group)

Orientation angles and axes are readily available

In [ ]:
O_au.angle

In [ ]:
O_au.axis

## Map properties

Map properties are stored in the `CrystalMap.prop` attribute dictionary

In [ ]:
xmap.prop

All properties in this dictionary are also available directly from the `CrystalMap` as attributes

In [ ]:
xmap.iq

In [ ]:
xmap.dp

We can add a map property by specifying its name and an initial value in each map point

In [ ]:
xmap.prop["grain_boundary"] = 0

xmap.grain_boundary

In [ ]:
xmap.prop["grain_boundary2"] = np.arange(xmap.size, dtype=int)

xmap.grain_boundary2

We can also delete a property from the `prop` dictionary

In [ ]:
del xmap.prop["grain_boundary2"]

xmap.prop

## Plotting

Map plotting can either be done via the [CrystalMap.plot()](../reference/generated/orix.crystal_map.CrystalMap.plot.rst) method, or via the [CrystalMapPlot](../reference/generated/orix.plot.CrystalMapPlot.rst) `matplotlib` projection. To plot a phase map via `CrystalMap.plot()`, we simply do

In [ ]:
xmap.plot()

Using the `matplotlib` projection

In [ ]:
# fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
# im = ax.plot_map(xmap)

Hover over figure points to display the (x,y) position and orientations in that point when plotting interactively!

Note that `plot()` wraps `matplotlib.axes.Axes.imshow`. All key word arguments in `plot()` are passed to `imshow()`, so be sure to check [its documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html) out for any additional arguments.

If we obtain a colour per orientation using [IPFColorKeyTSL.orientation2color()](../reference/generated/orix.plot.IPFColorKeyTSL.orientation2color.rst) (see also the [inverse pole figure tutorial](inverse_pole_figures.rst)), we can plot this as well

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(
    xmap.phases["austenite"].point_group, direction=Vector3d.zvector()
)
rgb_au = ckey_m3m.orientation2color(xmap["austenite"].orientations)
rgb_fe = ckey_m3m.orientation2color(xmap["ferrite"].orientations)

In [ ]:
xmap["austenite"].plot(rgb_au)

In [ ]:
xmap["ferrite"].plot(rgb_fe)

And the combined plot with the IPF color key added to the figure:

In [ ]:
rgb_all = np.zeros((xmap.size, 3))
rgb_all[xmap.phase_id == 1] = rgb_au
rgb_all[xmap.phase_id == 2] = rgb_fe

fig = xmap.plot(rgb_all, return_figure=True)

rc = {"font.size": 8}
with plt.rc_context(rc):  # Temporarily reduce font size
    ax_ipfkey = fig.add_axes(
        [0.72, 0.87, 0.2, 0.1],
        projection="ipf",
        symmetry=xmap.phases["austenite"].point_group,
    )
    ax_ipfkey.plot_ipf_color_key()
    ax_ipfkey.set_title("")

We can also color orientations from their Euler angles using [EulerColorKey.orientation2color()](../reference/generated/orix.plot.EulerColorKey.orientation2color.rst)

In [ ]:
ckey_euler = plot.EulerColorKey(xmap.phases["austenite"].point_group)
rgb_au_euler = ckey_euler.orientation2color(xmap["austenite"].orientations)
rgb_fe_euler = ckey_euler.orientation2color(xmap["ferrite"].orientations)

In [ ]:
rgb_all_euler = np.zeros((xmap.size, 3))
rgb_all_euler[xmap.phase_id == 1] = rgb_au_euler
rgb_all_euler[xmap.phase_id == 2] = rgb_fe_euler

xmap.plot(rgb_all_euler)

We can plot the color key to see the fundamental Euler region for point group *432*

In [ ]:
ckey_euler.plot()

We can add any overlay, from any property with a value in each map point, to the map by either passing the property name as a string, or the actual (flattened) array

In [ ]:
xmap.plot(overlay=xmap.dp)

To save our phase map with the scalebar and legend, but without white padding

In [ ]:
fig = xmap.plot(overlay="dp", return_figure=True, remove_padding=True)
fig.savefig(tempdir + "phase_map.png", bbox_inches="tight", pad_inches=0)

To save phase map without a scalebar, legend and white padding, and one image pixel per map point

In [ ]:
ax = fig.axes[0]
ax

In [ ]:
# 2D NumPy array, possibly with an RGB tuple in each element
plt.imsave(tempdir + "phase_map_no_fluff.png", arr=ax.images[0].get_array())

We can plot any property with a value in each map point, also adding a colorbar

In [ ]:
fig = xmap.plot(
    xmap.dp,
    cmap="inferno",
    colorbar=True,
    colorbar_label="Dottproduct",
    return_figure=True,
)

We can update the colorbar

In [ ]:
cbar = fig.axes[0].colorbar
cbar

In [ ]:
cbar.ax.set_ylabel("Dot product");

We can also plot orientation related values, like axis and angles etc., and restrict the color bar maximum

In [ ]:
# Get rotation angles in degrees
angles = xmap.rotations.angle * 180 / np.pi

xmap.plot(
    angles,
    vmax=angles.max() - 10,
    overlay=xmap.iq,
    colorbar=True,
    colorbar_label=r"Rotation angle, $\omega$ [$\degree$]",
)

To plot only one phase, while passing custom
* scalebar properties (https://github.com/ppinard/matplotlib-scalebar/)
* legend properties (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html)

In [ ]:
xmap["austenite"].plot(
    scalebar_properties=dict(location="upper left", frameon=False, sep=6),
    legend_properties=dict(
        framealpha=1, handlelength=1.5, handletextpad=0.1
    ),
)

Plot only a rectangle of the map

In [ ]:
xmap2 = xmap[20:50, 40:90]
xmap2.plot(overlay=xmap2.dp)

Plot only parts of a map based on chained conditionals, like belonging to one phase or having a property value above a threshold

In [ ]:
# Conditional slicing
xmap[xmap.dp > 0.81].plot(
    "iq", cmap="gray", colorbar=True, colorbar_label="Image quality, $Q$"
)

# Chained conditional slicing
xmap[(xmap.dp > 0.81) & (xmap.phase_id == 1)].plot(
    "dp", cmap="viridis", colorbar=True, colorbar_label="Dot product"
)

Plot histogram of a property per phase

In [ ]:
# Property of interest
this_prop = "dp"

# Plot phase map again to see color changes
xmap.plot(overlay=this_prop, remove_padding=True)

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in xmap.phases_in_data:
    labels.append(p.name)
    colors.append(p.color)

    # Accessing the property dictionary directly
    data.append(xmap[p.name].prop[this_prop])
    # or indirectly
    # data.append(xmap[p.name].dp)

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data, bins=20, histtype="bar", density=True, label=labels, color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();

In [ ]:
# Remove files written to disk in this tutorial
import os

for f in [
    tempdir + "sdss_ferrite_austenite2.h5",
    tempdir + "sdss_dp_ci.ang",
    tempdir + "phase_map.png",
    tempdir + "phase_map_no_fluff.png",
]:
    os.remove(f)
os.rmdir(tempdir)